# Augmentation Visualization

This notebook visualizes sample images from all three datasets and shows how they appear after different augmentation strategies.

**Datasets:**
- **Main Dataset**: 4 samples (PV domain)
- **Plant_doc Dataset**: 4 samples (PV domain)  
- **FieldPlant Dataset**: 4 samples (Field domain)

**Augmentation Strategies:**
1. **Original Images** (no augmentation)
2. **Baseline Augmentation** (from 5_fine_tune_models.ipynb)
3. **Quick Wins Augmentation** (from 5.1_fine_tune_models_quick_wins.ipynb)
4. **Progressive Adaptation Augmentation** (same as Quick Wins)

## Important Notes:

**For PV Domain Images (Main & Plant_doc):**
- All approaches use the **SAME minimal augmentation**: Resize + RandomHorizontalFlip
- **Note**: For visualization, we add small rotation and color jitter to make augmentations visible
- In actual training, PV images only use Resize + RandomHorizontalFlip (minimal augmentation)
- Differences between approaches are in **SAMPLING STRATEGY**, not augmentation

**For Field Domain Images (FieldPlant):**
- **Baseline**: Mild augmentation (Resize + Flip + Light ColorJitter)
- **Quick Wins / Progressive**: Aggressive augmentation (rotation, perspective, blur, sharpness, erasing, etc.)
- You should see **clear visual differences** for field images


In [25]:
# Imports and Setup (using same approach as 5.1_fine_tune_models_quick_wins.ipynb)
import os
from pathlib import Path
import json
import random
import numpy as np
import torch

from PIL import Image
import torchvision.transforms as T
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

# Paths - same as 5.1_fine_tune_models_quick_wins.ipynb
current_dir = Path(os.getcwd())

# Check if we're in experiment_2 subdirectory
if current_dir.name == "experiment_2":
    BASE_DIR = current_dir.parent
else:
    BASE_DIR = current_dir

METADATA_DIR = BASE_DIR / "metadata"
LABEL_MAPPING_PATH = METADATA_DIR / "label_mapping.json"
DATASET_INDEX_PATH = METADATA_DIR / "dataset_index.json"
DATA_DIR = BASE_DIR / "data"
PLANT_DOC_DIR = DATA_DIR / "Plant_doc"
FIELDPLANT_DIR = DATA_DIR / "FieldPlant_reformatted"  # Same as in 5.1 notebook

OUTPUT_DIR = BASE_DIR / "augmentation_visualizations"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Base directory: {BASE_DIR}")
print(f"Data directory: {DATA_DIR}")
print(f"FieldPlant directory: {FIELDPLANT_DIR}")
print(f"FieldPlant exists: {FIELDPLANT_DIR.exists()}")
print(f"Output directory: {OUTPUT_DIR}")


Base directory: d:\Programming\Seminar_Project\PLANT_LEAF_DISEASE_DETECTION
Data directory: d:\Programming\Seminar_Project\PLANT_LEAF_DISEASE_DETECTION\data
FieldPlant directory: d:\Programming\Seminar_Project\PLANT_LEAF_DISEASE_DETECTION\data\FieldPlant_reformatted
FieldPlant exists: True
Output directory: d:\Programming\Seminar_Project\PLANT_LEAF_DISEASE_DETECTION\augmentation_visualizations


In [26]:
# Load metadata (same as 5.1_fine_tune_models_quick_wins.ipynb)
with open(LABEL_MAPPING_PATH, "r") as f:
    label_mapping = json.load(f)

with open(DATASET_INDEX_PATH, "r") as f:
    dataset_index = json.load(f)

# Create mappings
id_to_label = {c["id"]: c["canonical_label"] for c in label_mapping["classes"]}
label_to_id = {v: k for k, v in id_to_label.items()}

# Plant_doc and FieldPlant mappings (same as 5.1 notebook)
plant_doc_to_canonical = {
    "Apple_leaf": "apple_healthy",
    "Apple_rust_leaf": "apple_cedar_apple_rust",
    "Apple_Scab_Leaf": "apple_apple_scab",
    "Bell_pepper_leaf": "pepper,_bell_healthy",
    "Bell_pepper_leaf_spot": "pepper,_bell_bacterial_spot",
    "Blueberry_leaf": "blueberry_healthy",
    "Cherry_leaf": "cherry_healthy",
    "Corn_Gray_leaf_spot": "corn_cercospora_leaf_spot_gray_leaf_spot",
    "Corn_leaf_blight": "corn_northern_leaf_blight",
    "Corn_rust_leaf": "corn_common_rust",
    "grape_leaf": "grape_healthy",
    "grape_leaf_black_rot": "grape_black_rot",
    "Peach_leaf": "peach_healthy",
    "Potato_leaf_early_blight": "potato_early_blight",
    "Potato_leaf_late_blight": "potato_late_blight",
    "Raspberry_leaf": "raspberry_healthy",
    "Soyabean_leaf": "soybean_healthy",
    "Squash_Powdery_mildew_leaf": "squash_powdery_mildew",
    "Strawberry_leaf": "strawberry_healthy",
    "Tomato_Early_blight_leaf": "tomato_early_blight",
    "Tomato_leaf": "tomato_healthy",
    "Tomato_leaf_bacterial_spot": "tomato_bacterial_spot",
    "Tomato_leaf_late_blight": "tomato_late_blight",
    "Tomato_leaf_mosaic_virus": "tomato_tomato_mosaic_virus",
    "Tomato_leaf_yellow_virus": "tomato_tomato_yellow_leaf_curl_virus",
    "Tomato_mold_leaf": "tomato_leaf_mold",
    "Tomato_Septoria_leaf_spot": "tomato_septoria_leaf_spot",
}

fieldplant_to_canonical = {
    "Corn___Gray_leaf_spot": "corn_cercospora_leaf_spot_gray_leaf_spot",
    "Corn___rust_leaf": "corn_common_rust",
    "Corn___leaf_blight": "corn_northern_leaf_blight",
    "Corn___healthy": "corn_healthy",
    "Tomato___healthy": "tomato_healthy",
    "Tomato___leaf_mosaic_virus": "tomato_tomato_mosaic_virus",
    "Tomato___leaf_yellow_virus": "tomato_tomato_yellow_leaf_curl_virus",
}

print(f"Loaded metadata for {len(label_mapping['classes'])} classes")
print(f"Plant_doc mappings: {len(plant_doc_to_canonical)} classes")
print(f"FieldPlant mappings: {len(fieldplant_to_canonical)} classes")


Loaded metadata for 39 classes
Plant_doc mappings: 27 classes
FieldPlant mappings: 7 classes


In [27]:
# Define augmentation transforms (NO normalization for clear visualization)
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]
IMG_SIZE = 224

# BASELINE AUGMENTATIONS (for visualization - NO normalization)
# For PV: Add visible rotation and color jitter to demonstrate augmentation is applied
# NOTE: In actual training, PV uses only Resize + RandomHorizontalFlip (minimal)
# We add rotation/color here just for visualization purposes
baseline_pv_basic = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE)),
    T.RandomHorizontalFlip(p=1.0),  # Always flip for visualization
    T.RandomRotation(degrees=15),  # Visible rotation to show augmentation is applied
    T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.1),  # Visible color change
])

baseline_field = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE)),
    T.RandomHorizontalFlip(p=1.0),  # Always flip for visualization
    T.ColorJitter(brightness=0.3, contrast=0.3),  # Mild color changes
])

# QUICK WINS / PROGRESSIVE ADAPTATION AUGMENTATIONS
# For PV: Same as baseline (same augmentation strategy)
# In training: Only Resize + RandomHorizontalFlip
# For visualization: Same rotation/color as baseline to show they're identical
quickwins_pv_basic = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE)),
    T.RandomHorizontalFlip(p=1.0),  # Always flip for visualization
    T.RandomRotation(degrees=15),  # Same rotation as baseline
    T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.1),  # Same color change as baseline
])

# Quick Wins field: AGGRESSIVE augmentation - make it very visible
quickwins_field_aggressive = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE)),
    T.RandomHorizontalFlip(p=1.0),  # Always flip
    T.RandomVerticalFlip(p=1.0),  # Always flip
    T.RandomRotation(degrees=30),  # Significant rotation
    T.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.2),  # Strong color changes
    T.RandomAffine(degrees=0, translate=(0.15, 0.15), scale=(0.7, 1.3)),  # More translation/scale
    T.RandomPerspective(distortion_scale=0.3, p=1.0),  # Always apply, stronger distortion
    T.GaussianBlur(kernel_size=5, sigma=(0.5, 1.0)),  # More blur
    T.RandomAdjustSharpness(sharpness_factor=3, p=1.0),  # Always apply, stronger sharpness
])

print("✓ Augmentation transforms defined (NO normalization for clear visualization)")


✓ Augmentation transforms defined (NO normalization for clear visualization)


In [28]:
# Data loading functions (same as 5.1_fine_tune_models_quick_wins.ipynb)
def load_plant_doc_data(data_dir, split="train"):
    """Load Plant_doc dataset entries and map to main dataset class IDs."""
    entries = []
    split_dir = data_dir / split
    
    if not split_dir.exists():
        print(f"Warning: {split_dir} does not exist!")
        return entries
    
    for folder in split_dir.iterdir():
        if not folder.is_dir():
            continue
        
        canonical = plant_doc_to_canonical.get(folder.name)
        if canonical is None:
            continue
        
        if canonical not in label_to_id:
            continue
        
        class_id = label_to_id[canonical]
        
        # Get all images in folder
        image_files = list(folder.glob("*.jpg")) + list(folder.glob("*.JPG"))
        for img_path in image_files:
            entries.append({
                "path": str(img_path),
                "class_id": class_id,
                "dataset": "plant_doc",
                "domain": "pv",
                "split": split
            })
    
    return entries

def load_fieldplant_data(data_dir):
    """Load FieldPlant dataset entries and map to main dataset class IDs."""
    entries = []
    
    if not data_dir.exists():
        print(f"Warning: {data_dir} does not exist!")
        return entries
    
    for folder in data_dir.iterdir():
        if not folder.is_dir():
            continue
        
        canonical = fieldplant_to_canonical.get(folder.name)
        if canonical is None:
            continue
        
        if canonical not in label_to_id:
            continue
        
        class_id = label_to_id[canonical]
        
        # Get all images in folder (same as 5.1 notebook)
        image_files = list(folder.glob("*.jpg")) + list(folder.glob("*.JPG"))
        for img_path in image_files:
            entries.append({
                "path": str(img_path),
                "class_id": class_id,
                "dataset": "fieldplant",
                "domain": "field",
                "split": "train"
            })
    
    return entries

print("✓ Data loading functions defined")


✓ Data loading functions defined


In [29]:
# Function to get sample images (using same data loading approach as 5.1)
def get_sample_images(dataset_name, num_samples=4):
    """Get sample images from a dataset."""
    samples = []
    
    if dataset_name == "main":
        # Get samples from main dataset
        train_entries = [e for e in dataset_index if e["split"] == "train"]
        random.seed(42)
        selected = random.sample(train_entries, min(num_samples, len(train_entries)))
        for entry in selected:
            img_path = Path(entry["path"])
            # Resolve path relative to BASE_DIR if it's a relative path (same as 5.1)
            if not img_path.is_absolute():
                img_path = BASE_DIR / img_path
            else:
                img_path = Path(img_path)
            if img_path.exists():
                samples.append({
                    "path": img_path,
                    "label": id_to_label[entry["class_id"]],
                    "domain": "pv"
                })
    
    elif dataset_name == "plant_doc":
        # Get samples from Plant_doc (same approach as 5.1)
        train_entries = load_plant_doc_data(PLANT_DOC_DIR, split="train")
        random.seed(42)
        selected = random.sample(train_entries, min(num_samples, len(train_entries)))
        for entry in selected:
            img_path = Path(entry["path"])
            # Resolve path (same as 5.1)
            if not img_path.is_absolute():
                img_path = BASE_DIR / img_path
            else:
                img_path = Path(img_path)
            if img_path.exists():
                samples.append({
                    "path": img_path,
                    "label": id_to_label[entry["class_id"]],
                    "domain": "pv"
                })
    
    elif dataset_name == "fieldplant":
        # Get samples from FieldPlant (same approach as 5.1)
        all_entries = load_fieldplant_data(FIELDPLANT_DIR)
        if len(all_entries) > 0:
            random.seed(42)
            selected = random.sample(all_entries, min(num_samples, len(all_entries)))
            for entry in selected:
                img_path = Path(entry["path"])
                # Resolve path (same as 5.1)
                if not img_path.is_absolute():
                    img_path = BASE_DIR / img_path
                else:
                    img_path = Path(img_path)
                if img_path.exists():
                    samples.append({
                        "path": img_path,
                        "label": id_to_label[entry["class_id"]],
                        "domain": "field"
                    })
        else:
            print(f"Warning: No FieldPlant entries found. Checked: {FIELDPLANT_DIR}")
    
    return samples

# Get sample images
main_samples = get_sample_images("main", 4)
plant_doc_samples = get_sample_images("plant_doc", 4)
fieldplant_samples = get_sample_images("fieldplant", 4)

print(f"Main dataset samples: {len(main_samples)}")
print(f"Plant_doc samples: {len(plant_doc_samples)}")
print(f"FieldPlant samples: {len(fieldplant_samples)}")


Main dataset samples: 4
Plant_doc samples: 4
FieldPlant samples: 4


In [30]:
# Function to apply augmentation and return PIL Image
def apply_augmentation_visual(img_pil, transform, apply_erasing=False):
    """Apply augmentation and return as PIL Image for visualization."""
    # Make a copy to avoid modifying original
    img_copy = img_pil.copy()
    # Apply transform (returns PIL Image, not tensor)
    augmented_img = transform(img_copy)
    
    # Apply RandomErasing if needed (requires tensor)
    if apply_erasing:
        # Convert to tensor, apply erasing, convert back
        tensor = T.ToTensor()(augmented_img)
        erasing = T.RandomErasing(p=1.0, scale=(0.05, 0.25), ratio=(0.3, 3.3))
        tensor_erased = erasing(tensor)
        # Convert back to PIL
        img_np = tensor_erased.permute(1, 2, 0).numpy()
        img_np = (img_np * 255).astype(np.uint8)
        augmented_img = Image.fromarray(img_np)
    
    return augmented_img

print("✓ Helper functions defined")


✓ Helper functions defined


In [31]:
# Create visualization function
def create_augmentation_comparison(samples, dataset_name, domain_type):
    """Create a comparison visualization for a dataset."""
    num_samples = len(samples)
    
    if num_samples == 0:
        print(f"No samples found for {dataset_name}")
        return
    
    # Create figure: rows = samples, cols = Original, Baseline, Quick Wins, Progressive, Aug List
    fig = plt.figure(figsize=(22, 4.5 * num_samples))
    gs = gridspec.GridSpec(num_samples, 5, figure=fig, hspace=0.35, wspace=0.2)
    
    for idx, sample in enumerate(samples):
        try:
            # Load original image (same path resolution as 5.1)
            img_path = sample["path"]
            if not Path(img_path).is_absolute():
                img_path = BASE_DIR / img_path
            else:
                img_path = Path(img_path)
            
            img_original = Image.open(img_path).convert("RGB")
            img_original_resized = img_original.resize((224, 224))
            
            # Determine which augmentation to use based on domain
            if domain_type == "pv":
                baseline_transform = baseline_pv_basic
                quickwins_transform = quickwins_pv_basic
                progressive_transform = quickwins_pv_basic  # Same as quickwins for PV
            else:  # field
                baseline_transform = baseline_field
                quickwins_transform = quickwins_field_aggressive
                progressive_transform = quickwins_field_aggressive  # Same as quickwins for field
            
            # Apply augmentations
            # For field images, also apply RandomErasing
            apply_erasing = (domain_type == "field")
            img_baseline = apply_augmentation_visual(img_original.copy(), baseline_transform, apply_erasing=False)
            img_quickwins = apply_augmentation_visual(img_original.copy(), quickwins_transform, apply_erasing=apply_erasing)
            img_progressive = apply_augmentation_visual(img_original.copy(), progressive_transform, apply_erasing=apply_erasing)
            
            # Debug: Check if images are different
            if idx == 0:
                baseline_array = np.array(img_baseline)
                quickwins_array = np.array(img_quickwins)
                progressive_array = np.array(img_progressive)
                original_array = np.array(img_original_resized)
                
                diff_baseline = np.abs(baseline_array.astype(float) - original_array.astype(float)).mean()
                diff_quickwins = np.abs(quickwins_array.astype(float) - original_array.astype(float)).mean()
                diff_progressive = np.abs(progressive_array.astype(float) - original_array.astype(float)).mean()
                
                print(f"\\nProcessing {sample['label']} (domain: {domain_type})")
                print(f"  Original image size: {img_original.size}")
                print(f"  Mean pixel difference from original:")
                print(f"    Baseline: {diff_baseline:.2f}")
                print(f"    Quick Wins: {diff_quickwins:.2f}")
                print(f"    Progressive: {diff_progressive:.2f}")
                
                # For PV, differences should be visible now (we added rotation/color for visualization)
                if domain_type == "pv":
                    if diff_baseline < 5 and diff_quickwins < 5:
                        print(f"  NOTE: Differences are small. This may indicate:")
                        print(f"        - Image is symmetric (flip doesn't change it)")
                        print(f"        - Rotation/color changes are subtle")
                        print(f"        In training, PV uses only Resize + Flip (minimal)")
                else:  # field
                    if diff_quickwins < 20:
                        print(f"  WARNING: Field augmentation differences seem small!")
                        print(f"           Expected larger differences for aggressive augmentation.")
            
            # Plot original
            ax1 = fig.add_subplot(gs[idx, 0])
            ax1.imshow(img_original_resized)
            title = f"Original\\n{sample['label']}"
            if idx == 0 and domain_type == "pv":
                title += "\\n(Minimal aug)"
            ax1.set_title(title, fontsize=10, fontweight='bold')
            ax1.axis('off')
            
            # Plot baseline
            ax2 = fig.add_subplot(gs[idx, 1])
            ax2.imshow(img_baseline)
            title = "Baseline"
            if idx == 0 and domain_type == "pv":
                title += "\\n(Resize+Flip)"
            elif idx == 0 and domain_type == "field":
                title += "\\n(Mild)"
            ax2.set_title(title, fontsize=10, fontweight='bold')
            ax2.axis('off')
            
            # Plot Quick Wins
            ax3 = fig.add_subplot(gs[idx, 2])
            ax3.imshow(img_quickwins)
            title = "Quick Wins"
            if idx == 0 and domain_type == "pv":
                title += "\\n(Same as baseline)"
            elif idx == 0 and domain_type == "field":
                title += "\\n(Aggressive)"
            ax3.set_title(title, fontsize=10, fontweight='bold')
            ax3.axis('off')
            
            # Plot Progressive
            ax4 = fig.add_subplot(gs[idx, 3])
            ax4.imshow(img_progressive)
            title = "Progressive"
            if idx == 0 and domain_type == "pv":
                title += "\\n(Same aug)"
            elif idx == 0 and domain_type == "field":
                title += "\\n(Same aug)"
            ax4.set_title(title, fontsize=10, fontweight='bold')
            ax4.axis('off')
            
            # Add augmentation list (only for first sample to save space)
            ax5 = fig.add_subplot(gs[idx, 4])
            ax5.axis('off')
            
            if idx == 0:  # Only show augmentation list for first sample
                if domain_type == "pv":
                    baseline_list = [
                        "BASELINE (PV):",
                        "• Resize (224x224)",
                        "• RandomHorizontalFlip (p=0.5)",
                        "• Normalize",
                        "",
                        "NOTE: PV images use minimal",
                        "augmentation (same for all).",
                        "In training: Only resize + flip.",
                        "For visualization: Added small",
                        "rotation & color jitter to show",
                        "augmentation is applied.",
                        "",
                        "Differences between approaches",
                        "are in SAMPLING strategy, not",
                        "augmentation."
                    ]
                    quickwins_list = [
                        "QUICK WINS (PV):",
                        "• Resize (224x224)",
                        "• RandomHorizontalFlip (p=0.5)",
                        "• Normalize",
                        "",
                        "Same augmentation as Baseline.",
                        "(Visualization shows rotation",
                        "and color jitter for clarity)"
                    ]
                    progressive_list = [
                        "PROGRESSIVE (PV):",
                        "• Resize (224x224)",
                        "• RandomHorizontalFlip (p=0.5)",
                        "• Normalize",
                        "",
                        "Same augmentation as others.",
                        "Difference: Progressive field",
                        "ratio (10% → 50%) and 5x",
                        "field weight in sampling."
                    ]
                else:  # field
                    baseline_list = [
                        "BASELINE (Field):",
                        "• Resize (224x224)",
                        "• RandomHorizontalFlip (p=0.5)",
                        "• ColorJitter (brightness=0.3,",
                        "  contrast=0.3)",
                        "• Normalize"
                    ]
                    quickwins_list = [
                        "QUICK WINS (Field):",
                        "• Resize (224x224)",
                        "• RandomHorizontalFlip (p=0.5)",
                        "• RandomVerticalFlip (p=0.3)",
                        "• RandomRotation (30°)",
                        "• ColorJitter (brightness=0.5,",
                        "  contrast=0.5, saturation=0.5,",
                        "  hue=0.2)",
                        "• RandomAffine (translate=0.15,",
                        "  scale=0.7-1.3)",
                        "• RandomPerspective (p=0.3)",
                        "• GaussianBlur (p=0.3)",
                        "• RandomAdjustSharpness (p=0.3)",
                        "• RandomErasing (p=0.4)",
                        "• Normalize"
                    ]
                    progressive_list = [
                        "PROGRESSIVE (Field):",
                        "• Same as Quick Wins",
                        "",
                        "Note: Same augmentation",
                        "strategy as Quick Wins.",
                        "Difference is in sampling:",
                        "• Progressive field ratio",
                        "  10% → 50% over epochs",
                        "• Field weight: 5x (vs 3x)"
                    ]
                
                text_content = "\\n".join(baseline_list)
                text_content += "\\n\\n" + "\\n".join(quickwins_list)
                text_content += "\\n\\n" + "\\n".join(progressive_list)
                
                ax5.text(0.05, 0.5, text_content, fontsize=8, verticalalignment='center',
                        family='monospace', transform=ax5.transAxes)
            
        except Exception as e:
            print(f"Error processing {sample['path']}: {e}")
            import traceback
            traceback.print_exc()
    
    plt.suptitle(f"{dataset_name.upper()} Dataset - Augmentation Comparison", 
                fontsize=16, fontweight='bold', y=0.995)
    
    # Save figure
    output_path = OUTPUT_DIR / f"{dataset_name.replace(' ', '_').lower()}_augmentation_comparison.png"
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    print(f"Saved: {output_path}")
    plt.close()

print("✓ Visualization function defined")


✓ Visualization function defined


In [32]:
# Create visualizations for each dataset
print("Creating augmentation visualizations...")
print("="*70)

if len(main_samples) > 0:
    print("\\n1. Main Dataset (PV domain)...")
    create_augmentation_comparison(main_samples, "Main Dataset", "pv")
else:
    print("\\n1. Main Dataset: No samples found")

if len(plant_doc_samples) > 0:
    print("\\n2. Plant_doc Dataset (PV domain)...")
    create_augmentation_comparison(plant_doc_samples, "Plant_doc", "pv")
else:
    print("\\n2. Plant_doc Dataset: No samples found")

if len(fieldplant_samples) > 0:
    print("\\n3. FieldPlant Dataset (Field domain)...")
    create_augmentation_comparison(fieldplant_samples, "FieldPlant", "field")
else:
    print("\\n3. FieldPlant Dataset: No samples found")
    print(f"   Checked directory: {FIELDPLANT_DIR}")
    if FIELDPLANT_DIR.exists():
        print(f"   Directory exists. Listing contents:")
        for item in list(FIELDPLANT_DIR.iterdir())[:10]:
            print(f"     - {item.name} ({'dir' if item.is_dir() else 'file'})")
    else:
        print(f"   Directory does not exist. Trying alternative paths...")
        alt_paths = [
            DATA_DIR / "FieldPlant",
            BASE_DIR / "data" / "FieldPlant",
            BASE_DIR / "data" / "FieldPlant_reformatted"
        ]
        for alt_path in alt_paths:
            if alt_path.exists():
                print(f"   Found alternative: {alt_path}")
                break

print("\\n" + "="*70)
print("✓ All augmentation visualizations created!")
print(f"\\nOutput directory: {OUTPUT_DIR}")


Creating augmentation visualizations...
\n1. Main Dataset (PV domain)...
\nProcessing tomato_spider_mites_two_spotted_spider_mite (domain: pv)
  Original image size: (256, 256)
  Mean pixel difference from original:
    Baseline: 41.31
    Quick Wins: 41.63
    Progressive: 33.96


Saved: d:\Programming\Seminar_Project\PLANT_LEAF_DISEASE_DETECTION\augmentation_visualizations\main_dataset_augmentation_comparison.png
\n2. Plant_doc Dataset (PV domain)...
\nProcessing tomato_septoria_leaf_spot (domain: pv)
  Original image size: (300, 300)
  Mean pixel difference from original:
    Baseline: 65.46
    Quick Wins: 63.40
    Progressive: 68.15
Saved: d:\Programming\Seminar_Project\PLANT_LEAF_DISEASE_DETECTION\augmentation_visualizations\plant_doc_augmentation_comparison.png
\n3. FieldPlant Dataset (Field domain)...
\nProcessing corn_northern_leaf_blight (domain: field)
  Original image size: (874, 1190)
  Mean pixel difference from original:
    Baseline: 47.86
    Quick Wins: 103.31
    Progressive: 89.51
Saved: d:\Programming\Seminar_Project\PLANT_LEAF_DISEASE_DETECTION\augmentation_visualizations\fieldplant_augmentation_comparison.png
\n======================================================================
✓ All augmentation visualizations created!
\nOutput directo